### **Data Inference**

#### **1. Import Library**

In [1]:
import pandas as pd
import numpy as np
import pickle

c:\ProgramData\Anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
c:\ProgramData\Anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


#### **2. Load model, scaler, and PCA**

In [2]:

# Memuat Scaler
with open('scaler_robust.pkl', 'rb') as file_1:
    scaler_robust = pickle.load(file_1)

with open('scaler_minmax.pkl', 'rb') as file_2:
    scaler_minmax = pickle.load(file_2)

# Memuat PCA
with open('pca_model.pkl', 'rb') as file_3:
    pca = pickle.load(file_3)

# Memuat model KMeans
with open('kmeans_model.pkl', 'rb') as file_4:
    km = pickle.load(file_4)

print("Model dan scaler telah dimuat")

Model dan scaler telah dimuat


### **3. Cluster Prediction**

In [3]:
# Seed untuk random state agar hasilnya dapat direproduksi
np.random.seed(42)

# Menentukan jumlah data dummy
n_samples = 10

In [4]:
# Membuat data dummy secara acak
data_dummy = {
    'BALANCE': np.random.rand(n_samples) * 5000,
    'BALANCE_FREQUENCY': np.random.rand(n_samples),
    'PURCHASES': np.random.rand(n_samples) * 2000,
    'ONEOFF_PURCHASES': np.random.rand(n_samples) * 1000,
    'INSTALLMENTS_PURCHASES': np.random.rand(n_samples) * 1000,
    'CASH_ADVANCE': np.random.rand(n_samples) * 1000,
    'PURCHASES_FREQUENCY': np.random.rand(n_samples),
    'ONEOFF_PURCHASES_FREQUENCY': np.random.rand(n_samples),
    'PURCHASES_INSTALLMENTS_FREQUENCY': np.random.rand(n_samples),
    'CASH_ADVANCE_FREQUENCY': np.random.rand(n_samples),
    'CASH_ADVANCE_TRX': np.random.randint(1, 10, n_samples),
    'PURCHASES_TRX': np.random.randint(1, 20, n_samples),
    'CREDIT_LIMIT': np.random.rand(n_samples) * 20000,
    'PAYMENTS': np.random.rand(n_samples) * 3000,
    'MINIMUM_PAYMENTS': np.random.rand(n_samples) * 1000,
    'PRC_FULL_PAYMENT': np.random.rand(n_samples)
}

In [5]:
# Buat DataFrame dari data dummy
data_inf = pd.DataFrame(data_dummy)

In [6]:
# Preprocessing data baru menggunakan scaler yang sudah dilatih
data_baru_scaled = scaler_robust.transform(data_inf)
data_baru_pca = pca.transform(scaler_minmax.transform(data_baru_scaled))

c:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


In [7]:
# Melakukan prediksi klaster untuk data baru
prediksi_klaster = km.predict(data_baru_pca)

In [8]:
# Menambahkan hasil prediksi ke dalam dataframe data baru
data_inf['prediction'] = prediksi_klaster

In [9]:
# Menampilkan hasil prediksi
data_inf.head()

,BALANCE,BALANCE_FREQUENCY,PURCHASES,ONEOFF_PURCHASES,INSTALLMENTS_PURCHASES,CASH_ADVANCE,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,CASH_ADVANCE_FREQUENCY,CASH_ADVANCE_TRX,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,prediction
0,1872.700594,0.020584,1223.705789,607.544852,122.038235,969.584628,0.388677,0.772245,0.863103,0.119594,7,2,4558.703251,2828.729112,284.840494,0.985650,2
1,4753.571532,0.969910,278.987721,170.524124,495.176910,775.132823,0.271349,0.198716,0.623298,0.713245,1,10,8542.155773,969.608796,36.886947,0.242055,1
2,3659.969709,0.832443,584.289297,65.051593,34.388521,939.498942,0.828738,0.005522,0.330898,0.760785,4,13,16360.295318,1556.371865,609.564334,0.672136,1
3,2993.292421,0.212339,732.723687,948.885537,909.320402,894.827350,0.356753,0.815461,0.063558,0.561277,4,6,17214.611665,2109.056877,502.679023,0.761620,1
4,780.093202,0.181825,912.139968,965.632033,258.779982,597.899979,0.280935,0.706857,0.310982,0.770967,5,12,139.042611,1090.888807,51.478751,0.237638,1


* Klaster ``2`` tampaknya lebih konsisten dalam melunasi tagihan penuh dengan ``PRC_FULL_PAYMENT`` yang sangat tinggi

* Klaster ``1`` memiliki variasi yang lebih besar dalam hal ``balance, purchases, cash advance, dan pembayaran.`` Mereka tidak konsisten dalam melunasi tagihan penuh, yang ditunjukkan oleh variasi besar dalam ``PRC_FULL_PAYMENT``

In [10]:
# Menghitung jumlah kemunculan setiap nilai unik dalam kolom prediction
data_inf['prediction'].value_counts()

prediction
1    8
2    1
0    1
Name: count, dtype: int64

* Klaster ``1`` memiliki mayoritas data poin yaitu ``8``, menunjukkan bahwa ``kebanyakan pelanggan memiliki karakteristik yang bervariasi`` dan mungkin memerlukan pengelolaan keuangan yang lebih baik, karena Klaster ``1`` memiliki balance yang lebih tinggi dibandingkan dengan klaster lain. Balance yang tinggi menunjukkan bahwa pelanggan dalam klaster ini memiliki saldo kredit yang besar yang belum dibayar

* Klaster ``2`` dan Klaster ``0`` masing-masing hanya memiliki satu data poin, menunjukkan adanya ``karakteristik yang unik dan berbeda`` dari mayoritas pelanggan 

In [11]:
# Menyimpan hasil prediksi ke dalam file CSV
data_inf.to_csv('inference_result.csv', index=False)
print("Hasil prediksi telah disimpan dalam 'inference_result.csv'.")

Hasil prediksi telah disimpan dalam 'inference_result.csv'.
